# Notebook lecture 12: Model Predictive Control
&copy; 2025 ETH Zurich, Ivan Robuschi, William Schnyder, Niclas Scheuer, Dejan Milojevic; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

Authors:
- Ivan Robuschi; irobuschi@ethz.ch
- William Schnyder; wschnyder@ethz.ch

In [ ]:
!pip install cvxpy ecos control matplotlib 

---

## Exericse 1 - From LQR to MPC


### 1. System Description and State-Space Matrices

We consider a continuous-time linearized model of an aircraft at altitude of 5000m and a speed of 128.2 m/sec:

$$
\dot x = A\,x + B\,u, \quad y = C\,x + D\,u,
$$

with states:
- $x_1$: angle of attack, 
- $x_2$: pitch angle, 
- $x_3$: pitch rate, 
- $x_4$: altitude, 

and control input $u$: elevator angle.

State-space representation:

$$
A = \begin{bmatrix}
-1.2822 & 0      & 0.98    & 0      \\
0       & 0      & 1.00    & 0      \\
-5.4293 & 0      & -1.8366 & 0      \\
-128.2  & 128.2  & 0       & 0
\end{bmatrix},
\quad
B = \begin{bmatrix}-0.3 \\ 0 \\ -17 \\ 0\end{bmatrix},
$$

$$
C = \begin{bmatrix} 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}, \quad 
D = \begin{bmatrix} 0 \\ 0 \end{bmatrix}, 
\quad T_s = 0.25~\mathrm{s}.
$$

There are some constraints that need to be satisfied:

- Elevator angle: $\pm0.262$ rad, ($\pm15^\circ$)  
- Elevator rate: $\pm0.524$ rad/s, ($\pm33^\circ$/s)  
- Pitch angle: $\pm0.349$ rad, ($\pm39^\circ$)

Run the cells below to setup the problem.

In [2]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import control

# System setup
A = np.array([[-1.2822,     0,      0.98,       0],
              [0,           0,      1,          0],
              [-5.4293,     0,      -1.8366,    0],
              [-128.2,      128.2,  0,          0]])
B = np.array([[-0.3], [0], [-17], [0]])
C = np.array([[0, 1, 0, 0], [0, 0, 0, 1]])
D = np.zeros((2,1))

# Discretize system
Ts = 0.25
sys_c = control.ss(A, B, C, D)
sys_d = control.c2d(sys_c, Ts)
Ad, Bd = sys_d.A, sys_d.B

# Constraints (MPC)
u_max = 0.262
# u_rate_max = 0.349 * Ts
# x2_max = 0.349
u_rate_max = 0.524 * Ts
x2_max = 0.524

# Additional parameters
T_sim = int(10 / Ts)
nx, nu = Ad.shape[0], Bd.shape[1]

# Prepare scenarios (different I.C. and horizons)
x0 = np.array([0, 0, 0, 10])
x0_high = np.array([0, 0, 0, 100])
N_def, N_small = 10, 3

# Plotting function scenarios
def plot_scenario(ax_state, ax_input, title, run):
    # Simulate
    xs, us = run()
    
    # Time vectors
    t  = np.arange(T_sim+1) * Ts
    tu = np.arange(len(us)) * Ts

    # State plot 
    ax1 = ax_state
    ax1.plot(t, xs[:, 3], 'b', label='alt')
    ax1.set_ylabel('alt (m)', color='b')
    ax1.tick_params(labelcolor='b')
    ax1.grid(True)

    ax1r = ax1.twinx()
    ax1r.plot(t, xs[:, 1], 'g', label='pitch')
    ax1r.set_ylabel('pitch (rad)', color='g')
    ax1r.tick_params(labelcolor='g')
    ax1r.axhline( x2_max,  color='g', linestyle='--')
    ax1r.axhline(-x2_max,  color='g', linestyle='--')

    # Combine legends
    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax1r.get_legend_handles_labels()
    ax1.legend(h1 + h2, l1 + l2, loc='upper right')
    ax1.set_title(title)

    # Input plot    
    ax2 = ax_input
    ax2.step(tu, us.flatten(), where='post')
    ax2.axhline( u_max,  color='r', linestyle='--')
    ax2.axhline(-u_max,  color='r', linestyle='--')
    ax2.set_ylabel('u (rad)')
    ax2.set_xlabel('t (s)')
    ax2.set_ylim(-0.5, 0.5)
    ax2.grid(True)

### 2. LQR vs. MPC: Formulations and Motivation
#### 2.1 LQR Formulation

We solve the infinite-horizon optimal control problem:

$$
\begin{aligned}
    J_\infty^*(x(k))
    &=\;\min\sum_{k=0}^\infty \bigl(x_k^T Q x_k + u_k^T R u_k\bigr) \\
    \text{subject to}\quad
    &\begin{cases}
    x_{k+1} = Ax_k + Bu_k,\\
    x_0 = x(k).
    \end{cases}
\end{aligned}
$$

with state-weighting $Q$ and input-weighting $R$.  
The resulting state feedback law is

$$
u_k = K_{LQR}x_k,
$$

where $K_{LQR}$ and the cost-to-go $P_\infty$ satisfy the discrete Algebraic Riccati Equation.

Run the cell below to setup the LQR formulation.

In [3]:
# Weights for LQR
Q = np.eye(nx)
R = 10 * np.eye(nu)

# Compute LQR gain and P_inf matrix
K_lqr, P_inf, _ = control.dlqr(Ad, Bd, Q, R)

# LQR with saturation
def simulate_lqr(x0, K, T, u_max, saturate):
    # Initialize state and input arrays
    x = np.zeros((nx, T+1))
    u = np.zeros((nu, T))
    
    # initial condition
    x[:, 0] = x0
    
    for k in range(T):

        # Compute control input, saturate (yes/no), and propagate dynamics
        u_unsat = -K @ x[:, k]

        if saturate:
            u_temp = np.clip(u_unsat, -u_max, u_max) # input saturation
            if k > 0:
                delta_u = u_temp - u[:, k-1] # input rate saturation
                delta_u_sat = np.clip(delta_u, -u_rate_max, u_rate_max)
                u_temp = u[:, k-1] + delta_u_sat
        else:
            u_temp = u_unsat

        u[:, k] = u_temp
        x[:, k+1] = Ad @ x[:, k] + Bd @ u[:, k]
    
    return x.T, u.T

The standard LQR formulation does not allow explicit handling of input or state constraints. As a result, applying actuator saturations e.g. clipping $u_k$ can violate the optimality and stability guarantees of the Riccati solution. 

Run the cell below to observe how LQR control with a saturated controller can lead to instability.

In [ ]:
# Run and plot scenario 1 (LQR without input saturation)
run_lqr = lambda: simulate_lqr(x0, K_lqr, T_sim, u_max, saturate=False)
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharex=True)
plot_scenario(ax_state=axes[0], ax_input=axes[1], title="LQR without input saturation", run=run_lqr)
plt.tight_layout()
plt.show()

# Run and plot scenario 2 (LQR with input saturation)
run_lqr_sat = lambda: simulate_lqr(x0, K_lqr, T_sim, u_max, saturate=True)
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharex=True)
plot_scenario(ax_state=axes[0], ax_input=axes[1], title="LQR with input saturation", run=run_lqr_sat)
plt.tight_layout()
plt.show()

#### 2.2 MPC Formulation

Model Predictive Control (MPC) overcomes LQR’s inability to enforce hard constraints by solving a finite‐horizon optimal control problem online in a receding‐horizon fashion.  At each time step $k$:

1. Measure or estimate the current state $x_k$,
2. Find the optimal input sequence for the entire planning window $N$,
3. Apply only the first control input $u_0^\star$.

Formally, given the current state $x(k)$, MPC solves

$$
\begin{aligned}
    J^*(x(k))
    &=\;\min_{u}
      \sum_{k=0}^{N-1} \bigl(x_k^T Q x_k + u_k^T R u_k\bigr)
      + x_N^T P_f x_N, \\[6pt]
    \text{subject to}\quad
    &\begin{cases}
      x_{k+1} = Ax_k + Bu_k, & k = 0,\dots,N-1,\\
      x_k \in X, u_k \in U,\\
      x_N \in X_N,\\
      x_0 = x(k).
    \end{cases}
\end{aligned}
$$

Run the cell below to setup the MPC formulation.

In [5]:
# MPC optimization problem
def solve_mpc(x0, N, constr_u, constr_x, constr_xT):

    # Decision variables
    x = cp.Variable((nx, N+1))
    u = cp.Variable((nu, N))

    # Initialize cost and constraints
    cost = 0
    constraints = []

    # Define intial state constraint
    constraints += [x[:, 0] == x0]

    for k in range(N):

        # Cost function (objective)
        cost += cp.quad_form(x[:, k], Q) + cp.quad_form(u[:, k], R)
        # Dynamics constraints
        constraints += [x[:, k+1] == Ad @ x[:, k] + Bd @ u[:, k]]
        # Input constraints
        if constr_u:
            constraints += [cp.abs(u[:, k]) <= u_max]
            constraints += [cp.abs(u[:, k] - u[:, k-1]) <= u_rate_max]
        # State constraints
        if constr_x:
            constraints += [cp.abs(x[1, k]) <= x2_max] if x2_max is not None else []

    # Terminal cost and constraints
    if constr_xT:
        cost += cp.quad_form(x[:, N], P_inf)
        constraints += [ cp.abs(K_lqr @ x[:, N]) <= u_max]
        constraints += [ cp.abs(x[1 ,N]) <= x2_max]


    # Define problem and solve
    prob = cp.Problem(cp.Minimize(cost), constraints)
    prob.solve(solver=cp.ECOS, verbose=False)
    return u[:, 0].value.flatten()

# Simulate MPC over T_sim an plot trajectories
def simulate_mpc(title, x0, N, constr_u=False, constr_x=False, constr_xT=False):
    # Initialize state and input trajectories
    x = np.zeros((nx, T_sim+1))
    u = np.zeros((nu, T_sim))
    x[:, 0] = x0

    for k in range(T_sim):

        # Solve MPC problem, get first control input and propagate system
        u[:, k] = solve_mpc(x[:, k], N, constr_u, constr_x, constr_xT)
        x[:, k+1] = Ad @ x[:, k] + Bd @ u[:, k]

    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharex=True)
    plot_scenario(ax_state=axes[0], ax_input=axes[1], title=title, run=lambda: (x.T, u.T))
    plt.tight_layout()
    plt.show()

In the following sections we will explore, step by step, how the constraints and design choice affect the closed‐loop behaviour of the MPC. Let's start with the input constraints.

**MPC without constraints**

- In the so called unconstrained MPC, there are no input constraints and no state constraints. The controller still drives both altitude and pitch to their set-points and keeps the pitch-angle bound satisfied, even without constraints. Yet the elevator angle and rate exceed their physical limits. 

**MPC with all input constraints**

- After activating the given hard bounds on the elevator and its rate, the controller continues to achieve convergence while this time respecting all input limits.

Run the cell below to visualize those two scenarios.

In [ ]:
# Run and plot scenario 2 (MPC without constraints)
simulate_mpc("MPC without constraints", x0=x0, N=N_def)

# Run and plot scenario 3 (MPC with all input constraints)
simulate_mpc("MPC with all input constraints", x0=x0, N=N_def, constr_u=True)

Let's move on and change the initial condition to a deviation of 100m of the desired altitude and see what happens. 

**MPC with larger initial condition**

- Starting from an altitude deviation of 100m, the controller again drives both altitude and pitch to their targets. The more aggressive descent however forces the pitch angle to go lower than the acceptable limit specified in the task. The plot shows that it even goes below -0.8 rad which is approximately -45°.

**MPC with state constraints**

- By adding a hard bound on the pitch, we can see on the plot, that the constraint indeed gets activated, not allowing the pitch angle to go below/above a certain limit. 

Run the cell below to visualize those two scenarios.

In [ ]:
# Run and plot scenario 4 (larger the initial condition)
simulate_mpc("MPC with larger initial condition", x0=x0_high, N=N_def, constr_u=True)

# Run and plot scenario 5 (MPC with state constraints)
simulate_mpc("MPC with state constraints", x0=x0_high, N=N_def, constr_u=True, constr_x=True)

In these final two scenarios, we first shrink the prediction horizon to observe its effects, and then show the standard way how to solve the arised issue, ending with the complete formulation for the standard MPC.  

**MPC with short horizon**  
- Shrinking the horizon makes the controller short-sighted, meaning it optimizes only the near future and can overlook longer-term consequences. This decrease in the prediction horizon can lead to loss of stability properties, so the closed-loop may oscillate or even drift away.

**MPC with terminal cost and constraint**  
- Rather than re-analysing stability each time parameters change, we append a terminal cost and terminal constraint which approximate the infinite-horizon tail that we truncated when going from the infinite-horizon LQR formulation to this finite-horizon MPC one. And just like that, stability guarantees are restored. 

In [ ]:
# Run and plot scenario 6 (shrinking the horizon)
simulate_mpc("MPC with short horizon", x0=x0, N=N_small, constr_u=True, constr_x=True)

# Run and plot scenario 7 (adding terminal cost and constraint)
simulate_mpc("MPC with terminal cost and constraint", x0=x0, N=N_small, constr_u=True, constr_x=True, constr_xT=True)

---

## Exericse 2 - 